In [1]:

import matplotlib.pyplot as plt
import cv2
import numpy as np
import glob

from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint, plot_skeleton_kpts

import torch
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset


c:\Users\EHmin\anaconda3\envs\EHmin\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Make Data

In [2]:
proper_review = glob.glob('./data/review_img/*/O/*')
proper_review = sorted(proper_review)
len(proper_review)

42526

In [3]:
unfit_review = glob.glob('./data/review_img/*/X/*')
unfit_review = sorted(unfit_review)
len(unfit_review)

3896

# Custom dataset

In [4]:
class CustomDataset(Dataset):
    def __init__(self,review_img_path,transform=None):
        self.review_img_path = review_img_path
        self.transform = transform
        
    def __getitem__(self,index):
        image = cv2.imread(self.review_img_path[index])
        image = letterbox(image, 960, stride=64, auto=True)[0]
        image = transforms.ToTensor()(image)
        image = torch.tensor(np.array([image.numpy()]))
        return image
    
    def __len__(self):
        return len(self.review_img_path)

In [5]:
dataset = CustomDataset(np.array(proper_review))
img_loader = DataLoader(dataset, batch_size = 1, shuffle=False, num_workers=0)

# Set model

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
weigths = torch.load('yolov7-w6-pose.pt', map_location=device)
model = weigths['model']
_ = model.float().eval()

if torch.cuda.is_available():
    model.half().to(device)

In [7]:
def prdict(model, image_lodaer):
    # model.half().to(device)
    model.eval()
    
    proper = 1
    unproper = 0

    flag = True
    
    with torch.no_grad():
        for img in image_lodaer:
            flag = True
            img = img[0]
            
            img = img.half().to(device)
            output, _ = model(img)
            output = non_max_suppression_kpt(output, 0.25, 0.65, nc=model.yaml['nc'], nkpt=model.yaml['nkpt'], kpt_label=True)
            output = output_to_keypoint(output)
            
            if output.size == 0:
                unproper += 1
                continue
            
            for landmark_presence in output[0][15::3]:
                if landmark_presence < 0.1:
                    flag = False
                    break
                
            if flag:
                proper += 1
            else:
                unproper += 1

            print((proper/(unproper + proper))*100)

In [8]:
prdict(model,img_loader )

c:\Users\EHmin\anaconda3\envs\EHmin\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


50.0
33.33333333333333
25.0
20.0
16.666666666666664
14.285714285714285
12.5
11.11111111111111
10.0
9.090909090909092
8.333333333333332
7.6923076923076925
14.285714285714285
13.333333333333334
12.5
11.76470588235294
11.11111111111111
10.526315789473683
10.0
9.523809523809524
9.090909090909092
8.695652173913043
12.5
12.0
11.538461538461538
11.11111111111111
10.714285714285714
10.344827586206897
10.0
9.67741935483871
9.375
9.090909090909092
11.76470588235294
11.428571428571429
11.11111111111111
13.513513513513514
13.157894736842104
12.82051282051282
12.5
14.634146341463413
14.285714285714285
16.27906976744186
15.909090909090908
15.555555555555555
15.217391304347828
14.893617021276595
14.583333333333334
14.285714285714285
14.000000000000002
13.725490196078432
13.461538461538462
13.20754716981132
12.962962962962962
12.727272727272727
12.5
14.035087719298245
13.793103448275861
13.559322033898304
13.333333333333334
13.114754098360656
12.903225806451612
12.698412698412698
12.5
12.3076923076923

KeyboardInterrupt: 

In [ ]:
proper = 1
unproper = 0

flag = True

for img_path in unfit_review:
    flag = True
    image = cv2.imread(img_path)
    image = letterbox(image, 960, stride=64, auto=True)[0]
    image = transforms.ToTensor()(image)
    image = torch.tensor(np.array([image.numpy()]))
    
    if torch.cuda.is_available():
        image = image.half().to(device)   
        output, _ = model(image)
        
    output = non_max_suppression_kpt(output, 0.25, 0.65, nc=model.yaml['nc'], nkpt=model.yaml['nkpt'], kpt_label=True)
    
    with torch.no_grad():
        output = output_to_keypoint(output)
        
    if output.size == 0:
        unproper += 1
        continue
    
    for landmark_presence in output[0][15::3]:
        if landmark_presence < 0.8:
            flag = False
            break
        
    if flag:
        proper += 1
    else:
        unproper += 1
    
    print((unproper/(unproper + proper))*100)